<a href="https://colab.research.google.com/github/Juliadambros/Sistema-de-Recomendacao-de-Filmes/blob/main/Recomenda%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Recomendação com SVD
Este notebook implementa um sistema de recomendação baseado em **filtragem colaborativa**, utilizando o algoritmo **SVD** da biblioteca `Surprise`.

## Importação das Bibliotecas

In [1]:
!pip install numpy==1.24.4
!pip uninstall -y scikit-surprise
!pip install scikit-surprise

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse, mae
import pandas as pd

##  Carregar o Dataset
O dataset deve conter pelo menos as colunas `userId`, `movieId` e `rating`.

In [3]:
url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.data"
df = pd.read_csv(url, sep='\t', names=['userId', 'itemId', 'rating', 'timestamp'])
df.head()

,userId,itemId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Carregando os dados dos filmes (para mostrar os nomes depois)

In [13]:
movie_url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.item"
movie_columns = ['itemId', 'title', 'release_date', 'video_release_date', 'IMDb_URL'] + [f'genre_{i}' for i in range(19)]
movies_df = pd.read_csv(movie_url, sep='|', names=movie_columns, encoding='latin-1')

# Criando o dicionário de ID -> título do filme
movie_titles = dict(zip(movies_df['itemId'], movies_df['title']))

## Preparação dos Dados
Define o intervalo de notas e carrega os dados no formato da biblioteca `Surprise`.

In [14]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'itemId', 'rating']], reader)

## Divisão dos Dados
Dividimos em 80% para treino e 20% para teste.

In [39]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=100)

## Treinamento do Modelo
Utilizamos o algoritmo **SVD**, que identifica padrões latentes nas avaliações.

In [40]:
model = SVD()
model.fit(trainset)

## Avaliação do Modelo
Utilizamos RMSE e MAE como métricas de erro.

In [41]:
predictions = model.test(testset)
print("Avaliação do Modelo:")
rmse(predictions)
mae(predictions)

Avaliação do Modelo:
RMSE: 0.9367
MAE:  0.7379


0.737907412590689

## Geração de Recomendações
Selecionamos um usuário e recomendamos filmes que ele ainda não avaliou, com base nas maiores notas previstas.

In [42]:
user_id = 78
user_movies = df[df['userId'] == user_id]['itemId'].unique()
all_movie_ids = df['itemId'].unique()
movies_not_seen = [movie for movie in all_movie_ids if movie not in user_movies]

user_predictions = [model.predict(user_id, movie_id) for movie_id in movies_not_seen]
top_10 = sorted(user_predictions, key=lambda x: x.est, reverse=True)[:10]

## Exibir Títulos dos Filmes

In [43]:
print(f"\nTop 10 recomendações para o usuário {user_id}:\n")
for pred in top_10:
    movie_name = movie_titles.get(int(pred.iid), f"ID {pred.iid}")
    print(f"{movie_name} — Nota prevista: {pred.est:.2f}")



Top 10 recomendações para o usuário 78:

Usual Suspects, The (1995) — Nota prevista: 4.75
Close Shave, A (1995) — Nota prevista: 4.73
Casablanca (1942) — Nota prevista: 4.60
Wallace & Gromit: The Best of Aardman Animation (1996) — Nota prevista: 4.60
Rear Window (1954) — Nota prevista: 4.60
Schindler's List (1993) — Nota prevista: 4.56
Shawshank Redemption, The (1994) — Nota prevista: 4.55
12 Angry Men (1957) — Nota prevista: 4.54
Amadeus (1984) — Nota prevista: 4.51
Manchurian Candidate, The (1962) — Nota prevista: 4.48
